<a href="https://colab.research.google.com/github/Wuuuuu05/Mask/blob/main/Mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import json
import sqlite3
import datetime

# 利用 requests 對 API 來源發送一個請求
url = "https://raw.githubusercontent.com/kiang/pharmacies/master/json/points.json"
response = requests.get(url)
# 將請求回應的內容存成一個字串格式
response_text = response.text
# 將長得像 json 格式的字串解析成字典或列表
data = json.loads(response_text)

med_count = {}
for d in data['features']:
    conunty = d['properties']['county']
    if conunty not in med_count:
      med_count[conunty] = 1
    else:
      med_count[conunty] += 1
print(med_count)

mask_count = {}
for d in data['features']:
    conunty = d['properties']['county']
    adult = d['properties']['mask_adult']
    if conunty not in mask_count:
      mask_count[conunty] = 0
    else:
      mask_count[conunty] += adult
# 將結果從大到小排列
mask_count = dict(sorted(mask_count.items(), key=lambda item: item[1]))
print(mask_count)


conn = sqlite3.connect('example.db')
c = conn.cursor()

# 新增且清空資料表
c.execute('''CREATE TABLE IF NOT EXISTS pharmacies
             (city text, counts text, createdAt datetime)''')
c.execute('''DELETE FROM pharmacies''')
conn.commit()

for city,counts in med_count.items():
  t = datetime.datetime.now()
  c.execute(f"INSERT INTO pharmacies VALUES ('{city}', {counts}, '{t}')")
  conn.commit()

# 查詢資料
c.execute("SELECT * FROM pharmacies")
print(c.fetchall())

conn.commit()
conn.close()

